In [68]:
import tensorflow as tf
import os
import shutil

In [69]:
from transformers import TFBertModel, BertConfig
configuration = BertConfig()
model = TFBertModel(configuration)
configuration = model.config

In [70]:
import pandas as pd

In [71]:
df = pd.read_excel("~/Desktop/data.xlsx")

In [72]:
df= df.dropna()
df = df.drop_duplicates()

In [73]:
labels = ["description","explanation","specification","process","advice","definition","distinction","identification","summary","list","justification", "example","comparison","categorization","narrative","cons","pros", "argument","elaboration","historical"]
def get_label(string):
    if isinstance(string, float):
        print("HELLO HERE")
        print(string)
        return 0
    for label in labels:
        if label.lower() in string.lower():
            return label
    print(string)
    
for s in df["label"].to_list():
    get_label(s)

In [74]:
df["label2"] = df["label"].apply(get_label)

In [75]:
df.head()

,string,question,label,label2
0,Most apples are a little sweet and a little ta...,what does an apple taste like,A description and explanation of how apples taste,description
1,Apples rank among the world's most popular fru...,does apple taste good?,a summary of different uses for apples,summary
2,"As white light passes through our atmosphere, ...",Why is the sky blue,explanation of why the sky is blue,explanation
3,"In other words, the color of the ocean and the...",IS the sky blue because of the ocean?,a distinction between why the ocean is blue an...,distinction
4,Color of Sunlight as seen on Earth's surface d...,Why does the sun appear to be yellow even if t...,explanation of why the sun is yellow,explanation


In [76]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
#shutil.rmtree(remove_dir)

84131840/84125825 [==============================] - 24s 0us/step


In [79]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [80]:
model_name = "bert-base-uncased"
max_length = 512

In [81]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [82]:
dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers","footers", "quotes"))
target_names=dataset.target_names
news_text = dataset.data
labels = dataset.target
(train_texts,valid_texts,train_labels,valid_labels)=train_test_split(news_text, labels, test_size=0.3)
from transformers import BertTokenizerFast, BertForSequenceClassification

In [83]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)


Ignored unknown kwarg option direction


In [86]:
model=BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=16,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    logging_steps=100,
    evaluation_strategy="steps",
    output_dir = "output_dir"
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [87]:
trainer = Trainer( model=model,args=training_args,train_dataset=train_datasetataset)
trainer.train()

NameError: name 'train_datasetataset' is not defined